# Initialization

In [1]:
import os
import shutil
import numpy as np

# Run simple simulation
## Load module

In [2]:
import PyRAMSES

## Load simulator

In [3]:
# ki till 0.7
# kp till 0.7
# td from 0.010 to 0.100

for ki in np.arange(0.10, 0.75, 0.1):
    ki = "{0:.2f}".format(round(ki,2))
    for kp in np.arange(0.10, 0.75, 0.1):
        kp = "{0:.2f}".format(round(kp,2))
        for td in np.arange(0.100, 0.101, 0.010):
            td = "{0:.2f}".format(round(td,2))

            print("kp = " + str(kp))
            print("ki = " + str(ki))
            print("td = " + str(td))

            ram = PyRAMSES.sim()

            # Load saved test-case
            case = PyRAMSES.cfg('cmd.txt')

            # Add one observation more
            case.addRunObs('MS g2') # will plot in real-time the voltage on bus g1

            # Run simulation and pause at t=15 seconds
            start_time=15
            ram.execSim(case,start_time)
            comp_type = ['TOR']
            comp_name = ['g14']
            prm_name = ['Tm0']
            ram.getPrm(comp_type, comp_name, prm_name)

            # Initialization
            comp_type = ['SYN']
            comp_name = ['g2']
            obs_name = ['Omega']
            errSum = 0.0
            t=240
            nominal_frequency = 1.0
            list_of_gens = ['g14', 'g15', 'g16', 'g17', 'g18']

            # PI Control
            for i in np.arange(start_time+1,t):
                #print("i = " + str(i))      
                actual_frequency = ram.getObs(comp_type,comp_name, obs_name)[0] # g2
                error = nominal_frequency - actual_frequency
                if abs(error)<0.00001:
                    error = 0.0
                #print("error = " + str(error))

                errSum += error * 1.0
                #print("errSum = " + str(errSum))
                output = float(kp) * float(error) + float(ki) * float(errSum)

                if abs(output)<0.00001:
                    output = 0.0
                # loop to send measurements to generators g14-g18

                for gen in list_of_gens:
                    command = 'CHGPRM TOR ' + gen + ' Tm0 ' + str(output/5.0) + ' 0'
                    #print(str(ram.getSimTime()+0.01)+' '+command)
                    td = float(td)
                    ram.addDisturb(ram.getSimTime() + td, command)
                ram.contSim(i)

            # Kill gnuplot
            os.system("TASKKILL /F /IM gnuplot.exe /T")
            print("kill successfully")

            # End simulation and exit
            ram.endSim()
            print("end simulation and exit successfully")

            # Open, read and re-write contents to another file (in public folder)
            with open("temp_display.cur") as f:
                with open("temp_display_.cur", "w") as f1:
                    for line in f:
                        if "error" not in line:
                            f1.write(line)

            # Copy the file (in public folder) to another prepared folder
            shutil.copy("temp_display_.cur", '/Users/el17jg/Desktop/GitHub/Nordic-test-system/curfiles')
            print("copy successfully")

            td = "{0:.2f}".format(round(td,2))

            # Rename the file in new folder
            os.rename('/Users/el17jg/Desktop/GitHub/Nordic-test-system/curfiles/temp_display_.cur', 
                      r'/Users/el17jg/Desktop/GitHub/Nordic-test-system/curfiles/temp_display_' + str(kp) + '-' + str(ki) + '-' + str(td) + 's' + '.cur')
            print("rename successfully")

            # Delete the file in public folder
            os.unlink("temp_display_.cur")
            print("delete successfully\n")

kp = 0.10
ki = 0.10
td = 0.10
kill successfully
end simulation and exit successfully
copy successfully
rename successfully
delete successfully

kp = 0.20
ki = 0.10
td = 0.10
kill successfully
end simulation and exit successfully
copy successfully
rename successfully
delete successfully

kp = 0.30
ki = 0.10
td = 0.10
kill successfully
end simulation and exit successfully
copy successfully
rename successfully
delete successfully

kp = 0.40
ki = 0.10
td = 0.10
kill successfully
end simulation and exit successfully
copy successfully
rename successfully
delete successfully

kp = 0.50
ki = 0.10
td = 0.10
kill successfully
end simulation and exit successfully
copy successfully
rename successfully
delete successfully

kp = 0.60
ki = 0.10
td = 0.10
kill successfully
end simulation and exit successfully
copy successfully
rename successfully
delete successfully

kp = 0.70
ki = 0.10
td = 0.10
kill successfully
end simulation and exit successfully
copy successfully
rename successfully
delete succes